# Diversified Portfolio Notebook

## Set Up and Execute Alapaca SDK Request

### Importing the Required Libraries

In [ ]:
# Import Required Libraries

import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np

%matplotlib inline

### Loading the .env file

In [ ]:
# Make sure dotenv is loaded
load_dotenv()

### Alpaca API and Secret Key Variables

In [ ]:
# Create Alpaca key variables to call from .env() file
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

display(type(alpaca_api_key))
display(type(alpaca_secret_key))

### Create the ALpaca REST Object

In [ ]:
# Create our Alpaca REST object
alpaca = tradeapi.REST(
  alpaca_api_key,
  alpaca_secret_key,
  api_version="v2"
)

alpaca

### Set the Start and End Dates

In [ ]:
# Set the start and end dates for the information we want to pull from Alpaca
start_date = pd.Timestamp("2020-01-02", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-11-02", tz="America/New_York").isoformat()

### Set the Tickers for Alpaca to Fetch

In [ ]:
# Set the list of tickers we want the Alpaca request to fetch
tickers = ["VOO", "MSFT", "AAPL", "BND", "AMT", "VUG", "TSLA", "AMZN", "EDV", "PLD"]

### Set the Timeframe for the Alpaca Request

In [ ]:
# Set the timeframe of collection. We just need the a daily price point
timeframe = "1D"

### Create the Request for Alpaca Using get_barset()

In [ ]:
# Create the request of the data to fetch via the Alpaca SDK
diversified_prices_df = alpaca.get_barset(
  tickers,
  timeframe,
  limit=1000,
  start=start_date,
  end=end_date
).df

diversified_prices_df

## Calculate Closing Prices and Daily Prices/Plots

In [ ]:
# Create a new empty DF that that holds only the closing prices of each stock
diversified_closing_prices_df = pd.DataFrame()

### For Loop to Access Closing Prices Only

In [ ]:
# Using a for loop, for every ticker in the tickers list, select only the close price for each ticker in the prices_df DF
for ticker in tickers:
  diversified_closing_prices_df[ticker] = diversified_prices_df[ticker]["close"]

diversified_closing_prices_df

### Calculate Aggregate Daily Returns for Portfolio

In [ ]:
# Calculate the daily returns for of the Stock Portfolio
diversified_daily_returns_df = diversified_closing_prices_df.sum(axis=1).pct_change().dropna()
diversified_daily_returns_df

### Plot the Aggregated Daily Returns

In [ ]:
# Plot daily returns for the stocks portfolio
diversified_daily_returns_df.plot(figsize=(15,10), title="Daily Returns - Diversified Portfolio (Jan. 2020 - Nov. 2021)")

## Set Up and Run Monte Carlo Simulation for 10 Years

In [ ]:
# Set up Monte Carlo Simulation for a 10 year time period with a $10,000 initial investment
MC_diversified_ten_year = MCSimulation(
  portfolio_data=diversified_prices_df,
  num_simulation=1000,
  num_trading_days=252 * 10
)

MC_diversified_ten_year.portfolio_data.head()

### Run the MC Simulation for 10 Years

In [ ]:
# Run the Monte Carlo simulation to forecast 10 years cumulative returns
MC_diversified_ten_year.calc_cumulative_return()

### Plot the MC 10 Year Simulation Visualization

In [ ]:
# Visualize the 10-year Monte Carlo simulation by creating an
# overlay line plot for the diversified portfolio
MC_diversified_ten_year.plot_simulation()

### Plot the MC 10 Year Simulation Distribution Visualization 

In [ ]:
# Visualize the probability distribution of the 10-year Monte Carlo simulation 
# by plotting a histogram
MC_diversified_ten_year.plot_distribution()

### Generate MC 10 Year Simulation Summary Statistics

In [ ]:
# Generate summary statistics from the 10-year Monte Carlo simulation results
# Save the results as a variable
MC_diversified_ten_year_summary_statistics = MC_diversified_ten_year.summarize_cumulative_return()

# Review the 30-year Monte Carlo summary statistics
print(MC_diversified_ten_year_summary_statistics)

### Calculate the Confidence Intervals to the 95% for the 10 year Simulation on a $10,000 initial investment

In [ ]:
# Calculate the 10-year confidence interval lower and upper projections for a $10,000 initial investment
diversified_ten_year_lower_ci = round(MC_diversified_ten_year_summary_statistics[8]*10000, 2)
diversified_ten_year_upper_ci = round(MC_diversified_ten_year_summary_statistics[9]*10000, 2)

print(f"With a 95% confidence interval, an initial investment of $10,000 in our Stocks Portfolio for 5 years, will be in the range of ${diversified_ten_year_lower_ci} and ${diversified_ten_year_upper_ci}")

In [ ]:
# Set up Monte Carlo Simulation for a 10 year time period with a $10,000 initial investment in our Diversified Portfolio
MC_diversified_ten_year = MCSimulation(
  portfolio_data=diversified_prices_df,
  weights=[.1, .1, .1, .1, .1, .1, .1, .1, .1, .1],
  num_simulation=1000,
  num_trading_days=252 * 10
)

MC_diversified_ten_year.portfolio_data.head()

In [ ]:
# Run the Monte Carlo simulation to forecast 10 years cumulative returns for our Diversified Portfolio
MC_diversified_ten_year.calc_cumulative_return()